In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.9/454.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.7/189.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.0 MB/s eta 0:0

In [4]:
# Imports and working paths
from autogluon.tabular import TabularPredictor
import pandas as pd, numpy as np, os

BASE = "/content/drive/MyDrive/californiahousing"  # X_train, y_train, X_test, sample_submission live here
OUT  = "/content/drive/MyDrive/ag-california"      # where I’ll save models/leaderboard/submission
os.makedirs(OUT, exist_ok=True)

In [5]:
# Read competition files from Drive
X_train = pd.read_csv(f"{BASE}/X_train.csv")
y_train = pd.read_csv(f"{BASE}/y_train.csv")
X_test  = pd.read_csv(f"{BASE}/X_test.csv")
sample  = pd.read_csv(f"{BASE}/sample_submission.csv")

print("X_train:", X_train.shape, "| y_train:", y_train.shape, "| X_test:", X_test.shape)
display(sample.head(2))


X_train: (16451, 22) | y_train: (16451, 1) | X_test: (4113, 22)


,id,TARGET
0,0,314021
1,1,144397


In [6]:
# Detect an id column (present in both X_train and y_train, e.g., 'Id')
common_cols = [c for c in X_train.columns if c in y_train.columns]
id_col = common_cols[0] if common_cols else None

# The target is the column in y_train that's not the id
label_candidates = [c for c in y_train.columns if c != id_col]
assert len(label_candidates) >= 1, "Couldn't find a target column in y_train."
label = label_candidates[0]

# Merge features + target
if id_col:
    train_df = X_train.merge(y_train[[id_col, label]], on=id_col, how="left")
else:
    train_df = X_train.copy()
    train_df[label] = y_train[label].to_numpy()

# Keep a version without the id for modeling
drop_cols = [id_col] if id_col else []
train_simple = train_df.drop(columns=drop_cols)
test_simple  = X_test.drop(columns=drop_cols, errors="ignore")

print("train_df:", train_df.shape, "| label:", label, "| id:", id_col or "(none)")
display(train_df.head(3))


train_df: (16451, 23) | label: TARGET | id: (none)


,Longitude,Latitude,HouseAge,total_rooms,total_bedrooms,population,households,MedInc,ocean_proximity__1H OCEAN,ocean_proximity_INLAND,...,computed_sqft,RoomsPerHousehold,BedroomsPerHousehold,PopulationPerHousehold,DistToPacific,DistToSF,DistToLA,GeoCluster,BedroomRatio,TARGET
0,0.750952,-0.801072,0.423492,-0.499866,-0.394692,-0.222182,-0.435339,-0.343554,True,False,...,-0.264684,-0.502694,0.082570,0.612452,0.753207,0.778152,-1.011590,89,0.383924,165100.0
1,-1.250266,0.722512,-0.847152,-0.846614,-0.971622,-0.892308,-0.941683,1.742317,False,False,...,0.182274,0.166947,-0.424455,-0.042249,-1.327588,-1.249369,0.893586,45,-0.665719,273300.0
2,-1.618650,1.559082,-0.608907,-0.786048,-0.875065,-0.772233,-0.854655,-1.132378,False,True,...,-0.283949,-0.042659,-0.277033,0.204686,-1.255122,-0.978006,1.665230,38,-0.388479,75000.0


In [7]:
# I train a baseline AutoGluon regressor with a small time cap.
predictor = TabularPredictor(
    label=label,
    problem_type="regression",
    eval_metric="rmse",
    path=OUT,                      # models + logs saved here (Drive)
).fit(
    train_data=train_simple,       # features + TARGET
    presets="medium_quality_faster_train",
    time_limit=300,                # ~5 minutes is plenty for this dataset
    verbosity=2
)


Preset alias specified: 'medium_quality_faster_train' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       11.27 GB / 12.67 GB (89.0%)
Disk Space Avail:   58.54 GB / 107.72 GB (54.3%)
Presets specified: ['medium_quality_faster_train']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "/content/drive/MyDrive/ag-california"
Train Data Rows:    16451
Train Data Columns: 22
Label Column:       TARGET
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11548.52 MB
	Train Data (Original)  Memory Usage: 2.21 MB (0.0% o

[1000]	valid_set's rmse: 33551.4
[2000]	valid_set's rmse: 31800.5
[3000]	valid_set's rmse: 30978.4
[4000]	valid_set's rmse: 30560
[5000]	valid_set's rmse: 30265.7
[6000]	valid_set's rmse: 30147.4
[7000]	valid_set's rmse: 30039.2
[8000]	valid_set's rmse: 29940.3
[9000]	valid_set's rmse: 29880.9
[10000]	valid_set's rmse: 29843.6


	-29843.1462	 = Validation score   (-root_mean_squared_error)
	38.17s	 = Training   runtime
	3.26s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 256.13s of the 256.13s of remaining time.
	Fitting with cpus=1, gpus=0, mem=0.0/11.0 GB


[1000]	valid_set's rmse: 29685.9
[2000]	valid_set's rmse: 28868.9
[3000]	valid_set's rmse: 28585.9
[4000]	valid_set's rmse: 28479.1
[5000]	valid_set's rmse: 28444
[6000]	valid_set's rmse: 28397.4
[7000]	valid_set's rmse: 28381.6
[8000]	valid_set's rmse: 28373.8
[9000]	valid_set's rmse: 28369.2
[10000]	valid_set's rmse: 28363.3


	-28362.6968	 = Validation score   (-root_mean_squared_error)
	30.99s	 = Training   runtime
	2.66s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 221.04s of the 221.04s of remaining time.
	Fitting with cpus=2, gpus=0
	-38968.2148	 = Validation score   (-root_mean_squared_error)
	95.33s	 = Training   runtime
	0.38s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 119.15s of the 119.15s of remaining time.
	Fitting with cpus=1, gpus=0
	Ran out of time, early stopping on iteration 7264.
	-26503.4066	 = Validation score   (-root_mean_squared_error)
	119.28s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 299.84s of the -0.30s of remaining time.
	Ensemble Weights: {'CatBoost': 0.81, 'LightGBM': 0.19}
	-26392.5836	 = Validation score   (-root_mean_squared_error)
	0.02s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 300.

In [8]:
# I capture the leaderboard for my repo artifacts.
lb = predictor.leaderboard(silent=False)
lb_path = f"{OUT}/leaderboard.csv"
lb.to_csv(lb_path, index=False)
print("Saved leaderboard ->", lb_path)


                 model     score_val              eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2 -26392.583618  root_mean_squared_error       2.677228  150.284817                0.000600           0.015128            2       True          5
1             CatBoost -26503.406585  root_mean_squared_error       0.016256  119.279464                0.016256         119.279464            1       True          4
2             LightGBM -28362.696819  root_mean_squared_error       2.660373   30.990226                2.660373          30.990226            1       True          2
3           LightGBMXT -29843.146221  root_mean_squared_error       3.261069   38.166688                3.261069          38.166688            1       True          1
4      RandomForestMSE -38968.214843  root_mean_squared_error       0.382322   95.334594                0.382322          95.334594            1       True          

In [9]:
# Predict house values on the held-out competition test set
test_pred = predictor.predict(test_simple)   # returns a 1-D Series of predictions
print("test_pred:", test_pred.shape)
test_pred.head()


test_pred: (4113,)


,TARGET
0,261848.609375
1,113672.843750
2,289279.750000
3,104510.906250
4,95214.046875


In [10]:
# Use sample_submission to guarantee column names and row order
id_col    = sample.columns[0]                               # typically 'Id'
target_col = [c for c in sample.columns if c != id_col][0]  # e.g., 'TARGET'

submission = sample[[id_col]].copy()
submission[target_col] = test_pred.values

sub_path = f"{OUT}/submission.csv"
submission.to_csv(sub_path, index=False)
print("Saved submission ->", sub_path)
submission.head(3)


Saved submission -> /content/drive/MyDrive/ag-california/submission.csv


,id,TARGET
0,0,261848.609375
1,1,113672.843750
2,2,289279.750000
